In [2]:
import numpy as np
from itertools import chain

In [170]:
def gibbs(docs, num_topics, iterations=5000):
    num_docs = len(docs)
    
    corpus = list(set(chain(*docs)))
    num_words = len(corpus)
    corpus_lookup = dict(zip(corpus, range(num_words)))
    
    current_topics = [np.random.randint(0, num_topics, len(words))
                      for words in docs]
    
    topic_counts = np.zeros(num_topics)
    for doc_topics in current_topics:
        for topic in doc_topics:
            topic_counts[topic] += 1

    document_counts = np.zeros((num_docs, num_topics))    
    for d, topics in enumerate(current_topics):
        for t in range(num_topics):
            document_counts[d][t] = sum(topics == t)
            
    word_counts = np.zeros((num_topics, num_words))
    for d, doc in enumerate(docs):
        for w, word in enumerate(doc):
            topic = current_topics[d][w]
            word_id = corpus_lookup[word]
            word_counts[topic][word_id] += 1
            
    for _ in range(iterations):
        for d, doc in enumerate(docs):
            for w, word in enumerate(doc):
                topic = current_topics[d][w]
                document_counts[d][topic] -= 1
                word_counts[topic][corpus_lookup[word]] -= 1
                topic_counts[topic] -= 1

                probs = np.zeros(num_topics)
                for j in range(num_topics):
                    probs[j] =  1 / num_topics # calculate

                new_topic = np.where(np.random.multinomial(1, probs))[0][0]
                current_topics[d][w] = new_topic
                document_counts[d][new_topic] += 1
                word_counts[new_topic][corpus_lookup[word]] += 1
                topic_counts[new_topic] += 1

    return current_topics, document_counts, word_counts, topic_counts

In [171]:
docs = [
    [
        "where", "where", "where", "where",
        "something", "something",
        "what","what","what","what","what",
        "who", "who","who","who",
    ],
    [
        "where","where","where","where",
        "something","something","something","something",
        "chair","chair","chair","chair",
        "desk","desk","desk","desk",
    ]
] * 100

In [ ]:
gibbs(docs, 3)